## CS189/289A HW3 code part2
### Yicheng Chen
### 02/15/2017

In [67]:
%matplotlib inline
import scipy.io
import random
from random import shuffle
import matplotlib.pyplot as plt
import numpy as np
from skimage.feature import hog
import pandas as pd
import cv2

---
## (a) fit Gaussian distribution to each digit class

In [ ]:
mnist = scipy.io.loadmat('mnist/train.mat')

In [ ]:
training_X = mnist['trainX'][:, :-1]
training_y = mnist['trainX'][:, -1]

In [8]:
def normalize_row(X):
    Xn = np.zeros(X.shape)
    for i in range(X.shape[0]):
        x = X[i, :]
        Xn[i, :] = (x/(np.sqrt(x.dot(x))+1e-15))
    return Xn

In [ ]:
training_Xn = normalize_row(training_X)

In [ ]:
for digit in range(1, 2):
    training_digit = training_Xn[training_y==digit, :]
    mean = training_digit.mean(axis=0)
    cov = np.cov(training_digit.T)

---
## (b) Visualize the covariance matrix

In [ ]:
plt.imshow(cov)
plt.colorbar()
plt.show()

---
## (c) Classify the digits in the test set on the basis of posterior probabilities with two different approaches

### (i) LDA

In [14]:
class LDA:
    def __init__(self):
        self.Mean = 0
        self.Cov = 0
        self.P = 0
    
    def fit(self, X, y):
        N = len(y)
        labels = set(y)
        Cov = 0
        Mean = np.zeros([len(labels), X.shape[1]])
        P = np.zeros([len(labels)])
        for i, label in enumerate(labels):
            X_class = X[y==label, :]
            Nc = X_class.shape[0]
            P[i] = Nc / N
            Mean[i, :] = X_class.mean(axis=0)
            Cov += np.cov(X_class.T) * Nc
        Cov = Cov / N
        self.Mean = Mean
        self.Cov = Cov
        self.P = P
    
    def predict(self, X):
        PreMat = np.linalg.pinv(self.Cov)
        L = self.Mean.dot(PreMat).dot(X.T).T \
            - 1/2*np.diag(self.Mean.dot(PreMat).dot(self.Mean.T)) \
            - np.log(self.P)
        yp = np.argmax(L.T, axis=0)
        return yp
    
    def accuracy(self, X, y):
        yp = self.predict(X)
        N = len(y)
        Nerror = (yp != y).sum()
        return 1 - Nerror/N
        

In [ ]:
mnist = scipy.io.loadmat('mnist/train.mat')

In [ ]:
training = mnist['trainX']
shuffle(training)
validation = training[0:10000]
training = training[10000:]

In [ ]:
training_X = normalize_row(training[:, :-1])
training_y = training[:, -1]
validation_X = normalize_row(validation[:, :-1])
validation_y = validation[:, -1]

In [ ]:
subset_size = np.array([100, 200, 500, 1000, 2000, 5000, 10000, 30000, 50000])

In [ ]:
all_accuracy = np.zeros(len(subset_size))

In [ ]:
for i, N in enumerate(subset_size):
    idx = random.sample(range(50000), N)
    lda = LDA()
    lda.fit(training_X[idx, :], training_y[idx])
    all_accuracy[i] = lda.accuracy(validation_X, validation_y)

In [ ]:
plt.plot(subset_size, all_accuracy * 100)
plt.xscale('log')
plt.xlabel('Training set size')
plt.ylabel('Accuracy(%)')
plt.show()

### (ii) QDA

In [15]:
class QDA:
    def __init__(self, a):
        self.Mean = 0
        self.Cov = 0
        self.P = 0
        self.a = a
        
    def Q(self, X, m, C, p):
        PreMat = np.linalg.inv(C)
        Const = - 1/2*np.log(np.linalg.det(C)+1e-20) + np.log(p)
        Qc = np.array([(-1/2*(x-m).dot(PreMat).dot(x-m)) + Const for x in X])
        return Qc
    
    def fit(self, X, y):
        N = len(y)
        d = X.shape[1]
        labels = set(y)
        Cov = np.zeros([X.shape[1], d, len(labels)])
        Mean = np.zeros([len(labels), d])
        P = np.zeros([len(labels)])
        for i, label in enumerate(labels):
            X_class = X[y==label, :]
            Nc = X_class.shape[0]
            P[i] = Nc / N
            Mean[i, :] = X_class.mean(axis=0)
            Cov[:, :, i] = np.cov(X_class.T) + self.a*np.eye(d)
        self.Mean = Mean
        self.Cov = Cov
        self.P = P
    
    def predict(self, X):
        nC = len(self.P)
        L = np.zeros([nC, len(X)])
        for i in range(nC):
            L[i, :] = self.Q(X, self.Mean[i], self.Cov[:, :, i], self.P[i])
        yp = np.argmax(L, axis=0)
        return yp
    
    def accuracy(self, X, y):
        yp = self.predict(X)
        N = len(y)
        Nerror = (yp != y).sum()
        return 1-Nerror/N
        

In [ ]:
subset_size = np.array([100, 200, 500, 1000, 2000, 5000, 10000, 30000, 50000])
all_accuracy = np.zeros(len(subset_size))

In [ ]:
for i, N in enumerate(subset_size):
    idx = random.sample(range(50000), N)
    qda = QDA(1e-8)
    qda.fit(training_X[idx, :], training_y[idx])
    all_accuracy[i] = qda.accuracy(validation_X, validation_y)

In [ ]:
plt.plot(subset_size, all_accuracy * 100)
plt.xscale('log')
plt.xlabel('Training set size')
plt.ylabel('Accuracy(%)')
plt.show()

### (iii) which perform better?

(Written answer.) QDA

### (iv) Kaggle 

In [153]:
#Calculate the HOG feature
def hog_feature(data):
    data_hog = np.zeros(data.shape)
    for i in range(0, len(data)):
        feature = data[i, :]
        image = feature.reshape(28, 28)
        fd, hog_image = hog(image, orientations=8, pixels_per_cell=(4, 4), cells_per_block=(1, 1), visualise=True)
        data_hog[i, :] = hog_image.ravel()
    return data_hog

In [154]:
mnist = scipy.io.loadmat('mnist/train.mat')
test = scipy.io.loadmat('mnist/test.mat')
training_X = mnist['trainX'][:, :-1]
training_y = mnist['trainX'][:, -1]
test_X = test['testX']

In [155]:
training_Xh = normalize_row(hog_feature(training_X))
test_Xh = normalize_row(hog_feature(test_X))

In [156]:
training_Xcombined = np.concatenate((normalize_row(training_X), training_Xh), axis=1)
test_Xcombined = np.concatenate((normalize_row(test_X), test_Xh), axis=1)

In [157]:
std = np.std(training_Xcombined, axis=0)

In [158]:
# use the max N variance word to build model
# max_idx = std.argsort()[:]
# training_X_max = training_Xcombined[:, max_idx]
# test_X_max = test_Xcombined[:, max_idx]

In [159]:
lda = LDA()
lda.fit(training_Xcombined, training_y)

yp_lda = lda.predict(test_Xcombined)
lda.accuracy(training_Xcombined, training_y)

0.96156666666666668

In [65]:
df = pd.DataFrame({'Category': yp_lda})
df.index.rename('Id', inplace=True)
df.to_csv('mnist/mnist_predict_org+hog_lda.csv')

---
## (d) Spam

In [2]:
spam = scipy.io.loadmat('spam/spam_data.mat')

### Bag-of-Words feature

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
import glob

In [4]:
SPAM_DIR = 'spam/'
HAM_DIR = 'ham/'
TEST_DIR = 'test/'
NUM_TEST_EXAMPLES = 10000
spam_filenames = glob.glob('spam/' + SPAM_DIR + '*.txt')
ham_filenames = glob.glob('spam/' + HAM_DIR + '*.txt')
test_filenames = ['spam/' + TEST_DIR + str(x) + '.txt' for x in range(NUM_TEST_EXAMPLES)]

In [5]:
all_text = []
for file in spam_filenames+ham_filenames: # use only training set data to build BOG
    with open(file, "r", encoding='utf-8', errors='ignore') as f:
        all_text.append(f.read())

In [6]:
all_test_text = []
for file in test_filenames: # use only training set data to build BOG
    with open(file, "r", encoding='utf-8', errors='ignore') as f:
        all_test_text.append(f.read())

In [9]:
vectorizer = CountVectorizer(min_df=4) # min word length=4
training_X = normalize_row(vectorizer.fit_transform(all_text).toarray())
test_X = normalize_row(vectorizer.transform(all_test_text).toarray())

In [10]:
training_y = np.concatenate((np.ones(len(spam_filenames)), np.zeros(len(ham_filenames))))

In [11]:
std = np.std(training_X, axis=0)

In [12]:
# use the max 2000 variance word to build model
max_idx = std.argsort()[-2000:]
training_X_max = training_X[:, max_idx]
test_X_max = test_X[:, max_idx]

In [16]:
lda = LDA()
lda.fit(training_X_max, training_y)
lda.accuracy(training_X_max, training_y)

0.98886169943464686

In [17]:
# Kaggle submission
yp = lda.predict(test_X_max)
df = pd.DataFrame({'Category': yp})
df.index.rename('Id', inplace=True)
df.to_csv('spam/spam_BOW_predict.csv')

## (e) For Experts

In [18]:
# max 10 variance words
max_idx = std.argsort()[-10:]
training_X_max = training_X[:, max_idx]
test_X_max = test_X[:, max_idx]

In [19]:
lda = LDA()
lda.fit(training_X_max, training_y)
lda.accuracy(training_X_max, training_y)

0.7610328242342419

In [20]:
# min 10 variance words
min_idx = std.argsort()[:10]
training_X_min = training_X[:, min_idx]
test_X_min = test_X[:, min_idx]

In [21]:
lda = LDA()
lda.fit(training_X_min, training_y)
lda.accuracy(training_X_min, training_y)

0.49308075267909879